# Quiver Quant Alternative Data

#### Imports

In [1]:
#supressing Pandas future warnings when indexing on datetime : https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import requests
import quiverquant
from configparser import ConfigParser
from io import BytesIO
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')

parser = ConfigParser()
_ = parser.read('notebook.cfg')

nasdaq_auth_key = parser.get('my_api', 'nasd_key')
quant_auth_key = parser.get('my_api', 'quant_key')
alpha_auth_key = parser.get('my_api', 'alpha_key')

## Using QuiverQuant's API to Access Data

In [2]:
parser = ConfigParser()
_ = parser.read('notebook.cfg')
quant_auth_key = parser.get('my_api', 'quant_key')
url = "https://api.quiverquant.com/beta/historical/wikipedia/FB"
headers = {'accept': 'application/json',
'X-CSRFToken': 'TyTJwjuEC7VV7mOqZ622haRaaUr0x0Ng4nrwSRFKQs7vdoBcJlK9qjAS69ghzhFu',
'Authorization': f'Token {quant_auth_key}'}
r = requests.get(url, headers=headers)
df = df = pd.read_json(r.content)
df.head()

,Date,Ticker,Views,pct_change_week,pct_change_month
0,2016-10-17,FB,259,-29.343629,-30.888031
1,2016-10-24,FB,183,-29.343629,-30.888031
2,2016-10-31,FB,205,12.021858,-30.888031
3,2016-11-07,FB,180,-12.195122,-30.888031
4,2016-11-14,FB,179,-0.555556,-30.888031


## Using QuiverQuant Library to Access Data

In [3]:
#https://github.com/Quiver-Quantitative/python-api
token = quant_auth_key
quiver = quiverquant.quiver(token)

## Facebook - Data Exploration

#### WallStreetBets Data (Historical Daily)

In [22]:
quiver.wallstreetbets(ticker ='FB').head()

https://api.quiverquant.com/beta/historical/wallstreetbets/FB


,Date,Ticker,Mentions,Rank,Sentiment
0,2018-08-01,FB,4,8,-0.054825
1,2018-08-02,FB,10,6,0.299040
2,2018-08-03,FB,10,5,0.005050
3,2018-08-06,FB,19,2,0.225889
4,2018-08-07,FB,13,5,-0.198246


#### Twitter Data (Historical Daily)

In [27]:
quiver.twitter(ticker='FB').sort_values('Date', ascending=True).head()

,Date,Ticker,Followers,pct_change_day,pct_change_week,pct_change_month
478,2020-05-02,FB,13467018,0.006579,0.027705,-0.205672
477,2020-05-03,FB,13467293,0.002042,0.027883,-0.176472
476,2020-05-05,FB,13465836,-0.010819,0.015531,-0.113410
475,2020-05-06,FB,13464762,-0.007976,0.010473,-0.065120
474,2020-05-07,FB,13464234,-0.003921,-0.001255,0.008557


In [6]:
twitter_data = quiver.twitter(ticker='FB')

In [18]:
twitter_data[['Date', 'Ticker', 'Followers']].tail()

,Date,Ticker,Followers
474,2020-05-07,FB,13464234
475,2020-05-06,FB,13464762
476,2020-05-05,FB,13465836
477,2020-05-03,FB,13467293
478,2020-05-02,FB,13467018


#### Wikipedia Pageviews (Historical Weekly)

In [16]:
quiver.wikipedia(ticker='FB').head()

,Date,Ticker,Views,pct_change_week,pct_change_month
0,2016-10-17,FB,259.0,-29.343629,-30.888031
1,2016-10-24,FB,183.0,-29.343629,-30.888031
2,2016-10-31,FB,205.0,12.021858,-30.888031
3,2016-11-07,FB,180.0,-12.195122,-30.888031
4,2016-11-14,FB,179.0,-0.555556,-30.888031


#### Congress Trading (Historical)

In [10]:
quiver.congress_trading(ticker='FB', politician=False).tail()

,ReportDate,TransactionDate,Ticker,Representative,Transaction,Amount,House,Range
147,2016-04-28,2016-04-28,FB,Pat Roberts,Purchase,1001.0,Senate,"$1,001 - $15,000"
148,2016-04-04,2016-03-24,FB,Sheldon Whitehouse,Purchase,1001.0,Senate,"$1,001 - $15,000"
149,2016-03-25,2016-03-07,FB,John Hoeven,Purchase,50001.0,Senate,"$50,001 - $100,000"
150,2016-02-23,2016-02-16,FB,Sheldon Whitehouse,Purchase,1001.0,Senate,"$1,001 - $15,000"
151,2016-02-23,2016-01-28,FB,Sheldon Whitehouse,Purchase,1001.0,Senate,"$1,001 - $15,000"


#### OTC Short Percentages (Historical)

In [50]:
quiver.offexchange(ticker='FB').tail()

,Date,Ticker,OTC_Short,OTC_Total,DPI
2386,2012-05-24,FB,6344764,17180465,0.369301
2387,2012-05-23,FB,7021287,22748790,0.308644
2388,2012-05-22,FB,7705869,33565864,0.229575
2389,2012-05-21,FB,6433108,43305137,0.148553
2390,2012-05-18,FB,23641525,77969848,0.303214


In [51]:
otc_data = quiver.offexchange(ticker='FB')[['Date', 'DPI']].sort_values(by='Date', ascending=True)

## Function Combining Several Sentiment Indicators  

In [144]:
def alt_dataframe(ticker, start_date, end_date):
    wsb_data = quiver.wallstreetbets(ticker ='FB')
    wsb_data['Date'] = pd.to_datetime(wsb_data['Date'])
    wsb_data.set_index('Date', inplace=True)
    congress_data = quiver.congress_trading(ticker='FB', politician=False).sort_values(by='ReportDate', ascending=True)[['ReportDate', 'Transaction']]
    congress_data['Transaction'] = congress_data['Transaction'].map({'Purchase' : 1, 'Sale' : -1})
    congress_data['ReportDate'] = pd.to_datetime(congress_data['ReportDate'])
    congress_data.set_index('ReportDate', inplace=True)
    otc_data = quiver.offexchange(ticker='FB')[['Date', 'DPI']].sort_values(by='Date', ascending=True)
    otc_data['Date'] = pd.to_datetime(otc_data['Date'])
    otc_data.set_index('Date', inplace=True)
    df = otc_data.join(congress_data)
    df2 = df.join(wsb_data)
    df2 = df2[df2.index >= f'{start_date}']
    df2 = df2[df2.index <= f'{end_date}']
    df2['Transaction'].fillna(0, inplace=True)
    return df2

In [146]:
ticker = 'FB'
start_date = '2021-01-01'
end_date = '2021-12-10'
alt_dataframe(ticker, start_date, end_date).head()

https://api.quiverquant.com/beta/historical/wallstreetbets/FB


,DPI,Transaction,Ticker,Mentions,Rank,Sentiment
2021-01-04,0.319705,0.0,FB,22.0,67.0,0.010582
2021-01-05,0.346115,0.0,FB,37.0,44.0,0.074581
2021-01-06,0.325061,0.0,FB,98.0,25.0,-0.029316
2021-01-07,0.301306,0.0,FB,76.0,25.0,0.021668
2021-01-08,0.341858,0.0,FB,63.0,24.0,-0.074205
